<a href="https://colab.research.google.com/github/TtttttH/COM6911-Team-Juliet/blob/main/CNN/CNN_USCHAD_w64_o75.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import numpy as np
import pandas as pd
import re
import matplotlib.pyplot as plt
from collections import Counter
from sklearn.model_selection import train_test_split

from tqdm import tqdm
import numpy as np
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torch.utils.data import TensorDataset
import torch.optim as optim
import torch.utils.data as data
from torchvision import datasets
import torchvision.transforms as transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu") # use gpu

In [ ]:
# loading data
usc_raw_data = pd.read_csv('/content/drive/MyDrive/Industrial Team Project/USCHAD_rawdata.csv')

In [ ]:
# only features and label.
usc_unprocessed = usc_raw_data[['acc_x','acc_y','acc_z','gyro_x','gyro_y','gyro_z','label']]

In [ ]:
# window_size: size of time window
# step: overlapping
# data: dataset
def time_windows(window_size,overlapping,data):
  sigmentation_data_temp = []
  sigmentation_data = []
  sigmentation_label = []
  for i in range(0,len(data),overlapping):
    acc_x = data['acc_x'].values[i:i+window_size]
    acc_y = data['acc_y'].values[i:i+window_size]
    acc_z = data['acc_z'].values[i:i+window_size]
    gyro_x = data['gyro_x'].values[i:i+window_size]
    gyro_y = data['gyro_y'].values[i:i+window_size]
    gyro_z = data['gyro_z'].values[i:i+window_size]
    total_label = data['label'].values[i:i+window_size]
    label = Counter(total_label).most_common()[0][0]
    sigmentation_data_temp.append([acc_x,acc_y,acc_z,gyro_x,gyro_y,gyro_z])
    sigmentation_arr = np.asarray(sigmentation_data_temp)
    sig_size = sigmentation_arr.shape
    if sig_size[2] == window_size:
      sigmentation_arr.reshape(window_size,6)
      sigmentation_data.append(sigmentation_arr)
      sigmentation_label.append(label)
      sigmentation_data_temp = []
    else:
      sigmentation_data_temp = []

  sigmentation_data_arr = np.asarray(sigmentation_data)
  sigmentation_label_arr = np.asarray(sigmentation_label)

  return sigmentation_data_arr,sigmentation_label_arr

In [ ]:
# here use semi-non-overlapping
# signal segmentation and relevant labels
uci_readings,uci_labels = time_windows(64,16,usc_unprocessed)

In [ ]:
# in the model, requires 0-n label so make labels from 1-12 into 0-11
uci_labels = uci_labels-1

In [ ]:
# use train_test_split to generate train,val and test data
x_train_75_overlapping,x_val_test_75_overlapping,y_train_75_overlapping,y_val_test_75_overlapping = train_test_split(uci_readings,uci_labels,test_size = 0.3)
x_val_75_overlapping,x_test_75_overlapping,y_val_75_overlapping,y_test_75_overlapping = train_test_split(x_val_test_75_overlapping,y_val_test_75_overlapping,test_size = 0.3 )

In [ ]:
train_dataset_75_overlapping = TensorDataset(torch.from_numpy(x_train_75_overlapping).to(torch.float32),torch.from_numpy(y_train_75_overlapping).to(torch.float32))
val_dataset_75_overlapping = TensorDataset(torch.from_numpy(x_val_75_overlapping).to(torch.float32),torch.from_numpy(y_val_75_overlapping).to(torch.float32))
test_dataset_75_overlapping = TensorDataset(torch.from_numpy(x_test_75_overlapping).to(torch.float32),torch.from_numpy(y_test_75_overlapping).to(torch.float32))

In [ ]:
# DataLoader generation
train_loader_75_overlapping = data.DataLoader(dataset = train_dataset_75_overlapping,batch_size = 64,shuffle = True)
val_loader_75_overlapping = data.DataLoader(dataset = val_dataset_75_overlapping,batch_size = 64,shuffle = True)
test_loader_75_overlapping = data.DataLoader(dataset = test_dataset_75_overlapping,batch_size = 64,shuffle = False)

In [ ]:
 # now define a class of CNN model
class CNN_w64_o75(nn.Module):
  def __init__(self):
    super(CNN_w64_o75, self).__init__()
    self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.ReLU(),
            nn.BatchNorm2d(16),
            nn.MaxPool2d(kernel_size=2, stride=2))
    self.layer2 = nn.Sequential(
             nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
             nn.ReLU(),
             nn.BatchNorm2d(32),
             nn.MaxPool2d(2,2))
    self.layer3 = nn.Sequential(
            nn.Conv2d(32, 64, kernel_size=5, stride=1, padding=2),
            nn.ReLU()
    )
    self.drop_out = nn.Dropout()
    self.fc1 = nn.Linear(1536, 512)
    self.fc3 = nn.Linear(512,12)
  def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)
        out = self.layer3(out)
        out = out.reshape(out.size(0), -1)
        out = self.drop_out(out)
        out = self.fc1(out)
        out = self.fc3(out)
        return out
        
model_w64_o75 = CNN_w64_o75().to(device)

In [ ]:
epoches = 50
lr = 0.001
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model_w64_o75.parameters(),lr=lr, weight_decay=0.0001,momentum = 0.8)

In [ ]:
# training
loss_list = []
for epoch in range(epoches):
    acc = 0
    correct = 0
    total = 0
    loss_num = 0
    runtime_loss = 0
    for (imgs,labels) in train_loader_75_overlapping:
        imgs = imgs.reshape(len(imgs),1,24,16)     
        imgs = imgs.to(device)
        labels = labels.type(torch.LongTensor)
        labels = labels.to(device)
        optimizer.zero_grad()
        outputs = model_w64_o75(imgs)
        loss = criterion(outputs, labels)
        runtime_loss += loss.item()
        loss.backward()
        optimizer.step()
        total += labels.size(0)
        _,predicted = torch.max(outputs.data,1)
        correct += (predicted == labels).sum().item()
        loss_num+=1
    acc = 100 * correct / total
    epoch_loss = runtime_loss / loss_num
    loss_list.append(epoch_loss)
    loss_num = 0
    print("Epoch: ",epoch+1, "accuracy: ", acc,' %',"Loss: ",epoch_loss)

Epoch:  1 accuracy:  50.78883347029772  % Loss:  1.3186786260411245
Epoch:  2 accuracy:  72.27081519572346  % Loss:  0.7552888797540829
Epoch:  3 accuracy:  76.81830386565015  % Loss:  0.6298582161360269
Epoch:  4 accuracy:  78.64901589058037  % Loss:  0.5728977511538497
Epoch:  5 accuracy:  79.813348738436  % Loss:  0.539153265780859
Epoch:  6 accuracy:  80.7832192867546  % Loss:  0.5098338627461528
Epoch:  7 accuracy:  81.67253850597626  % Loss:  0.4887268434723239
Epoch:  8 accuracy:  82.17862867464586  % Loss:  0.4712320129692027
Epoch:  9 accuracy:  82.68146424416003  % Loss:  0.4550772648516079
Epoch:  10 accuracy:  82.96949626941571  % Loss:  0.4486745974550267
Epoch:  11 accuracy:  83.31448377989146  % Loss:  0.4346500667726416
Epoch:  12 accuracy:  83.52033717647251  % Loss:  0.42711275067415044
Epoch:  13 accuracy:  83.86939293589253  % Loss:  0.41958328958975033
Epoch:  14 accuracy:  84.21356679657941  % Loss:  0.40782922644406666
Epoch:  15 accuracy:  84.36816025646242  % L

In [ ]:
# test
correct_test = 0

total_test = 0

with torch.no_grad():

  for (imgs_test,labels_test) in test_loader_75_overlapping:

    imgs_test = imgs_test.reshape(len(imgs_test),1,24,16)

    imgs_test = imgs_test.to(device)

    labels_test = labels_test.type(torch.LongTensor)

    labels_test = labels_test.to(device)

    outputs_test = model_w64_o75(imgs_test)

    labels_test = labels_test.reshape(labels_test.shape[0],)

    predicted_test = torch.max(outputs_test.data,1)[1]
      
    total_test += labels_test.size(0)
      
    correct_test += (predicted_test == labels_test).sum().item()

    test_accuracy = 100*correct_test/total_test

print('Test accuracy is: ',test_accuracy)

Test accuracy is:  85.33822691893944
